In [2]:
#| default_exp ai_edit

In [ ]:
#| export

# Core imports
from dotenv import load_dotenv
from lisette import *

load_dotenv()

True

In [ ]:
#| export
class TextEditor:
    def __init__(self, text: str = ""):
        self.text = text
    
    def delete_line(
        self, 
        index: int  # Zero-based line number to delete
    ) -> str:
        "Delete a line from the current text"
        lines = self.text.split('\n')
        
        if index < 0 or index >= len(lines):
            raise IndexError(f"Line index {index} is out of range. Text has {len(lines)} lines.")
        
        lines.pop(index)
        self.text = '\n'.join(lines)
        return f"Deleted line {index}. Updated text has {len(lines)} lines."
    
    def replace_line(
        self, 
        index: int,         # Zero-based line number to replace
        new_content: str    # New text content for the line
    ) -> str:
        "Replace a line with new content"
        lines = self.text.split('\n')
        
        if index < 0 or index >= len(lines):
            raise IndexError(f"Line index {index} is out of range. Text has {len(lines)} lines.")
        
        lines[index] = new_content
        self.text = '\n'.join(lines)
        return f"Replaced line {index} with: '{new_content}'"
    
    def insert_line(
        self, 
        index: int,     # Zero-based line number for new line
        content: str    # Text content for the new line
    ) -> str:
        "Insert a new line at the specified index"
        lines = self.text.split('\n')
        
        if index < 0 or index > len(lines):
            raise IndexError(f"Line index {index} is out of range. Text has {len(lines)} lines.")
        
        lines.insert(index, content)
        self.text = '\n'.join(lines)
        return f"Inserted line at index {index}: '{content}'"
    
    def replace_text(
        self, 
        old_text: str,  # Text string to find
        new_text: str   # Text string to replace with
    ) -> str:
        "Replace all occurrences of old_text with new_text"
        if old_text not in self.text:
            return f"Text '{old_text}' not found in document."
        
        count = self.text.count(old_text)
        self.text = self.text.replace(old_text, new_text)
        return f"Replaced {count} occurrence(s) of '{old_text}' with '{new_text}'"

In [ ]:
#| export 
def apply_edit(text: str, instruction: str, model: str, temperature: float = 0.3) -> str:
    """Apply an edit instruction to the given text and return the result.
    
    The AI will use available editing tools to modify the text according to the instruction.
    Returns only the edited text, not the AI's chat response.
    
    Args:
        text: The text to edit
        instruction: Natural language instruction for the edit
        model: Model identifier (e.g., 'openai/gpt-4o-mini')
        temperature: Controls randomness (0=deterministic, 1=creative). Default 0.3
    """
    # Create a temporary editor instance
    editor = TextEditor(text)
    
    # Create chat with system prompt and all editor methods as tools
    chat = Chat(
        model,
        sp="You are a precise text editor. Execute editing instructions accurately using the provided tools.",
        temp=temperature,
        tools=[editor.delete_line, editor.replace_line, editor.insert_line, editor.replace_text]
    )
    
    try:
        chat([f'Current text:\n{editor.text}\n\nInstruction: {instruction}'], max_steps=1)
    except Exception as e:
        # If tool fails (IndexError, etc), return original text
        print(f"Warning: Edit failed - {e}")
        return text

    return editor.text

In [ ]:
# Test the apply_edit function
test_text = '''Hello guys!
My name is batman
I like pizza'''

# Test 1: Delete a line
result1 = apply_edit(test_text, 'Delete the first line', 'openai/gpt-4o-mini')
print(f"Test 1 - Delete first line:")
print(result1)
print()

# Test 2: Replace text
result2 = apply_edit(test_text, 'Replace "guys" with "everyone"', 'openai/gpt-4o-mini')
print(f"Test 2 - Replace text:")
print(result2)
print()

# Test 3: Replace a line
result3 = apply_edit(test_text, 'Replace the second line with "My name is Robin"', 'openai/gpt-4o-mini')
print(f"Test 3 - Replace line:")
print(result3)
print()

# Test 4: Insert a line
result4 = apply_edit(test_text, 'Insert a new line at the beginning that says "Welcome!"', 'openai/gpt-4o-mini')
print(f"Test 4 - Insert line:")
print(result4)

Test 1 - Delete first line:
My name is batman
I like pizza

Test 2 - Replace text:
Hello everyone!
My name is batman
I like pizza

Test 2 - Replace text:
Hello everyone!
My name is batman
I like pizza

Test 3 - Replace line:
Hello guys!
My name is Robin
I like pizza

Test 3 - Replace line:
Hello guys!
My name is Robin
I like pizza

Test 4 - Insert line:
Welcome!
Hello guys!
My name is batman
I like pizza
Test 4 - Insert line:
Welcome!
Hello guys!
My name is batman
I like pizza
